In [16]:
import dataiku

In [79]:
def get_container_usage(container_name):
    # returns projects with a list of recipes that use the container (and whether the usage is explicit or inherited)
    projects_using_container = {}
    client = dataiku.api_client()
    projects = client.list_projects()
    for project in projects:
        projectKey = project['projectKey']
        project = client.get_project(projectKey)
        recipes_using_container = []
        recipes = project.list_recipes()
        for recipe in recipes:
            try:
                container = recipe['params']['containerSelection']
                conf, mode = container['containerConf'], container['containerMode']
                if conf == container_name:
                    recipes_using_container.append((recipe['name'], conf, mode))
            except:
                pass
        if recipes_using_container != []:
            projects_using_container[projectKey] = recipes_using_container
    return projects_using_container
            
get_container_usage('Test1') # example usage

{'CONTAINERUSAGE': [('compute_output1', 'Test2', 'EXPLICIT_CONTAINER'),
  ('compute_output2', 'Test2', 'EXPLICIT_CONTAINER'),
  ('compute_output3', 'Test2', 'EXPLICIT_CONTAINER')]}

In [82]:
def transfer_container_usage(container1, container2):
    # converts every recipe that uses container1 to container2
    # returns projects and their recipes that were modified
    projects_modified = {}
    client = dataiku.api_client()
    projects = client.list_projects()
    for project in projects:
        projectKey = project['projectKey']
        project = client.get_project(projectKey)
        recipes_modified = []
        recipes = project.list_recipes()
        for recipe in recipes:
            try:
                container = recipe['params']['containerSelection']
                conf, mode = container['containerConf'], container['containerMode']
                if conf == container1:
                    # change the container from container1 to container2
                    container_params = {}
                    container_params['containerConf'], container_params['containerMode'] = container2, 'EXPLICIT_CONTAINER'
                    recipe_settings = project.get_recipe(recipe['name']).get_settings()
                    recipe_settings.raw_params['containerSelection'] = container_params
                    recipe_settings.save()
                    recipes_modified.append((recipe['name'], container2, 'EXPLICIT_CONTAINER'))
            except:
                pass
        if recipes_modified != []:
            projects_modified[projectKey] = recipes_modified
    return projects_modified

transfer_container_usage('Test1', 'Test2') # example usage

{'CONTAINERUSAGE': [('compute_output1', 'Test2', 'EXPLICIT_CONTAINER'),
  ('compute_output2', 'Test2', 'EXPLICIT_CONTAINER'),
  ('compute_output3', 'Test2', 'EXPLICIT_CONTAINER')]}

In [51]:
container2 = 'Test2'
project = client.get_project('CONTAINERUSAGE')
recipe = project.get_recipe('compute_output1')


{'msg': 'Updated recipe CONTAINERUSAGE.compute_output1'}

In [0]:
column_name = 'C'
df['month'] = df['Date'].apply(lambda x : datetime.strptime(x, '%m-%d-%Y').month)
avg = df.groupby('month').agg('count')[column_name].mean()
df['avg_count'] = avg
df

In [0]:
# Write recipe outputs
avg_count_col = dataiku.Dataset("avg_count_col")
avg_count_col.write_with_schema(avg_count_col_df)